In [57]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [58]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
# display all columns

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Step 1: get the svi nyc data**

In [60]:
file_path1 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_svi.csv'
file_path2 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid_svi.csv'
file_path3 = '/content/drive/My Drive/X999/bbl_cleaned.csv'

In [61]:
normal = pd.read_csv(file_path1)
covid = pd.read_csv(file_path2)
bbl = pd.read_csv(file_path3)

In [62]:
bbl.head()

,borough,block,lot,community board,census tract 2010,cb2010,schooldist,council district,postcode,firecomp,policeprct,healtharea,sanitboro,sanitsub,address,zonedist1,zonedist2,zonedist3,overlay1,overlay2,spdist1,ltdheight,splitzone,bldgclass,landuse,easements,ownertype,ownername,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,otherarea,areasource,numbldgs,numfloors,unitsres,unitstotal,lotfront,lotdepth,bldgfront,bldgdepth,ext,proxcode,irrlotcode,lottype,bsmtcode,assessland,assesstot,exempttot,yearbuilt,yearalter1,yearalter2,histdist,landmark,builtfar,residfar,commfar,facilfar,borocode,bbl,condono,tract2010,xcoord,ycoord,latitude,longitude,zonemap,zmcode,sanborn,taxmap,appbbl,appdate,plutomapid,version,sanitdistrict,healthcenterdistrict,firm07_flag,pfirm15_flag,dcpedited,building_category,building_type,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade
0,BK,8366,222,318.0,696.02,2002.0,22.0,46.0,11234.0,E323,63.0,8822.0,3.0,4E,6815 AVENUE N,R3-1,NaN,NaN,NaN,NaN,NaN,NaN,False,A5,1.0,0.0,NaN,"EAST 69 AVENUE N DEVELOPMENT, LLC",2241.0,1288.0,0.0,1288.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2,1,1.0,27.0,83.0,19.0,33.0,NaN,2.0,False,0.0,2.0,5918.0,5918.0,0.0,2019.0,0.0,0.0,NaN,NaN,0.57,0.50,0.0,1.0,3,3083660222,NaN,69602.0,1008419.0,165883.0,40.621954,-73.912938,23b,NaN,315 067,3.0,3.083660e+09,04/26/2019,1,20v5,18.0,35.0,NaN,NaN,NaN,single-family,post-war,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","2009–present, post-financial crisis",single-unit,True,very small,Q1 (smallest 25%),2010-2020
1,BK,2571,28,301.0,561.00,1005.0,14.0,33.0,11222.0,L106,94.0,100.0,3.0,1A,87 CALYER STREET,M1-2/R6B,NaN,NaN,NaN,NaN,MX-8,NaN,False,A5,1.0,0.0,NaN,85 CALYER STREET LLC,1862.0,3478.0,0.0,3478.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,3,1,1.0,18.0,100.0,18.0,60.0,N,3.0,False,5.0,2.0,51000.0,66780.0,0.0,2018.0,0.0,2017.0,NaN,NaN,1.87,2.00,2.0,2.0,3,3025710028,NaN,561.0,995995.0,204223.0,40.727214,-73.957625,12c,NaN,304 033,30902.0,3.025710e+09,05/09/2019,1,20v5,1.0,30.0,NaN,NaN,NaN,single-family,post-war,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","2009–present, post-financial crisis",single-unit,True,medium-small,Q4 (largest 25%),2010-2020
2,BK,3197,8,304.0,429.00,1002.0,32.0,34.0,11237.0,E218,83.0,3200.0,3.0,1B,109 WILSON AVENUE,R6,NaN,NaN,NaN,NaN,NaN,NaN,False,S4,4.0,0.0,NaN,SERLIN BUILDING LIMITED PARTNERSHIP,2500.0,4125.0,1375.0,2750.0,0.0,1375.0,0.0,0.0,0.0,0.0,2.0,1.0,3,4,5.0,25.0,100.0,25.0,55.0,N,0.0,False,3.0,5.0,158850.0,381150.0,45280.0,1931.0,2001.0,0.0,NaN,NaN,1.65,2.43,0.0,4.8,3,3031970008,NaN,429.0,1004619.0,194842.0,40.701450,-73.926539,13b,NaN,309 037,31102.0,NaN,NaN,1,20v5,4.0,34.0,NaN,NaN,NaN,primarily_res_with_mixed_use,pre-war,False,low-rise,"Pre-1947, pre-rent-control","1931–1950, Manhattan Modern","1930-1945, great depression and WWII",3-5 units,False,medium-small,Q4 (largest 25%),1930-1939
3,QN,52,7,402.0,7.00,1000.0,30.0,26.0,11101.0,L115,108.0,720.0,4.0,2A,11-43 45 AVENUE,M1-4/R6A,NaN,NaN,NaN,NaN,LIC,NaN,False,C1,2.0,0.0,NaN,"TRIBECA TREASURES, LLC",2500.0,7416.0,380.0,7036.0,0.0,0.0,0.0,0.0,0.0,380.0,2.0,1.0,5,7,7.0,25.0,100.0,25.0,100.0,N,0.0,False,5.0,0.0,10350.0,977850.0,923360.0,1958.0,2007.0,2007.0,NaN,NaN,2.97,3.00,2.0,3.0,4,4000520007,NaN,7.0,998601.0,211689.0,40.747702,-73.948207,9b,NaN,401 019,40101.0,NaN,NaN,1,20v5,2.0,41.0,NaN,NaN,NaN,walk-up,post-war,False,mid-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",6-20 units,True,medium,Q4 (largest 25%),1950-1959
4,BK,6714,55,314.0,534.00,3000.0,21.0,44.0,11230.0,L156,70.0,7310.0,3.0,4D,1081 EAST 12 STREET,R5,R7A,NaN,NaN,NaN,NaN,NaN,True,C3,2.0,0.0,NaN,"RAMBOD, SHAHROKH",3500.0,2112.0,0.0,2112.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2,4,4.0,35.0,100.0,22.0,48.0,N,0.0,False,5.0,5.0,55350.0,255600.0,2390.0

In [63]:
bbl['key'] = bbl['borough'] + bbl['block'].astype(str) + bbl['lot'].astype(str)

In [64]:
bbl.shape

(752619, 98)

In [65]:
bbl.drop_duplicates(subset=['key'], inplace=True)
bbl.shape

(739983, 98)

In [66]:
bbl.bbl.nunique()

739983

In [67]:
30376/739983

0.04104959167980886

In [68]:
normal.bin.nunique(), bbl.key.nunique(), covid.bin.nunique()

(30376, 739983, 4381)

In [69]:
analysis_columns = normal.columns[-17:]
analysis_columns

Index(['average_year_eviction_count_per_unit(building)',
       'average_year_eviction_count(building)',
       'average_year_eviction_count_per_nta(nta)',
       'evictions_per_nta_1kpop(nta)', 'buildings_affected_per_nta',
       'unit_per_nta', 'evictions_per_1kunit_nta_1kpop(nta)',
       'children_impacted_nta_per_1kpop', 'poverty_impacted_nta_per_1kpop',
       'disability_impacted_nta_per_1kpop',
       'unemployed_impacted_nta_per_1kpop', 'elderly_impacted_nta_per_1kpop',
       'bh_impacted_nta_per_1kpop', 'hburd_impacted_nta_per_1kpop',
       'average_year_eviction_per_building_nta(nta)',
       'average_5year_eviction_count_per_unit(building)', 'bh_pct'],
      dtype='object')

In [70]:
# bbl.groupby('zipcode')

In [71]:
bbl.rename(columns={'postcode': 'zipcode'}, inplace=True)

In [72]:
bbl_building_count_zipcode = bbl.groupby('zipcode').agg({'bbl': 'count'}).sort_values('bbl', ascending=False)

In [73]:
bbl_building_count_zipcode.reset_index(inplace=True)

In [74]:
bbl_building_count_zipcode.rename(columns={'bbl': 'building_count_zipcode'}, inplace=True)

In [75]:
bbl_building_count_zipcode

,zipcode,building_count_zipcode
0,10314.0,18820
1,11234.0,17944
2,10312.0,17443
3,10306.0,14785
4,11236.0,13354
...,...,...
177,10280.0,4
178,11697.0,4
179,10282.0,3
180,10044.0,2


In [76]:
bbl_building_unit_count_zipcode = bbl.groupby('zipcode').agg({'unitsres': 'sum'}).sort_values('unitsres', ascending=False)

In [77]:
bbl_building_unit_count_zipcode

,unitsres
zipcode,
10467.0,47531
10025.0,43274
11226.0,40443
11385.0,37050
11235.0,34178
...,...
11040.0,783
10282.0,764
10044.0,389


In [78]:
bbl_building_unit_count_zipcode.reset_index(inplace=True)
bbl_building_unit_count_zipcode.rename(columns={'unitsres': 'unit_count_zipcode'}, inplace=True)
bbl_building_unit_count_zipcode

,zipcode,unit_count_zipcode
0,10467.0,47531
1,10025.0,43274
2,11226.0,40443
3,11385.0,37050
4,11235.0,34178
...,...,...
177,11040.0,783
178,10282.0,764
179,10044.0,389
180,10004.0,260


In [79]:
bbl_building_unit_count_zipcode = bbl_building_unit_count_zipcode.merge(bbl_building_count_zipcode, on='zipcode', how='left')
bbl_building_unit_count_zipcode

,zipcode,unit_count_zipcode,building_count_zipcode
0,10467.0,47531,4007
1,10025.0,43274,1290
2,11226.0,40443,4721
3,11385.0,37050,13351
4,11235.0,34178,6868
...,...,...,...
177,11040.0,783,551
178,10282.0,764,3
179,10044.0,389,2
180,10004.0,260,12


In [80]:
bbl_building_unit_count_zipcode.zipcode.dtype

dtype('float64')

In [81]:
normal = pd.merge(normal, bbl_building_unit_count_zipcode, on='zipcode', how='left')
covid = pd.merge(covid, bbl_building_unit_count_zipcode, on='zipcode', how='left')

In [82]:
# stop

In [83]:
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0


In [84]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0


In [85]:
analysis_columns = normal.columns[-19:]
analysis_columns

Index(['average_year_eviction_count_per_unit(building)',
       'average_year_eviction_count(building)',
       'average_year_eviction_count_per_nta(nta)',
       'evictions_per_nta_1kpop(nta)', 'buildings_affected_per_nta',
       'unit_per_nta', 'evictions_per_1kunit_nta_1kpop(nta)',
       'children_impacted_nta_per_1kpop', 'poverty_impacted_nta_per_1kpop',
       'disability_impacted_nta_per_1kpop',
       'unemployed_impacted_nta_per_1kpop', 'elderly_impacted_nta_per_1kpop',
       'bh_impacted_nta_per_1kpop', 'hburd_impacted_nta_per_1kpop',
       'average_year_eviction_per_building_nta(nta)',
       'average_5year_eviction_count_per_unit(building)', 'bh_pct',
       'unit_count_zipcode', 'building_count_zipcode'],
      dtype='object')

# **Zipcode specific analysis columns**

# **Average year eviction count per zipcode**

In [86]:
zipcode_evi_count = normal.groupby('zipcode').agg({'primary_key': 'count'}).sort_values('primary_key', ascending=False)
zipcode_evi_count.reset_index(inplace=True)
zipcode_evi_count.rename(columns={'primary_key': 'average_year_eviction_count_per_zipcode(z)'}, inplace=True)
zipcode_evi_count

,zipcode,average_year_eviction_count_per_zipcode(z)
0,10458,2221
1,10467,2192
2,10453,1994
3,10456,1880
4,10457,1765
...,...,...
171,10162,3
172,10044,2
173,11005,1
174,11359,1


In [87]:
zipcode_evi_count_covid = covid.groupby('zipcode').agg({'primary_key': 'count'}).sort_values('primary_key', ascending=False)
zipcode_evi_count_covid.reset_index(inplace=True)
zipcode_evi_count_covid.rename(columns={'primary_key': 'average_year_eviction_count_per_zipcode(z)'}, inplace=True)
zipcode_evi_count_covid

,zipcode,average_year_eviction_count_per_zipcode(z)
0,11226,175
1,10467,160
2,10458,158
3,11212,138
4,10453,136
...,...,...
165,10069,1
166,11366,1
167,11364,1
168,11357,1


In [88]:
normal = normal.merge(zipcode_evi_count, on='zipcode', how='left')
covid = covid.merge(zipcode_evi_count_covid, on='zipcode', how='left')

In [89]:
normal.head()

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.9140,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.931540,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.635941,-74.011883,7.0,38.0,118.0,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),1920.0,B2,2.0,2.0,"A.R.M. PARKING, LLC",1204.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,0.300000,0.6,38.0,0.408567,136,272.0,1.502085,0.103776,0.153213,0.034320,0.030643,0.053522,0.174050,0.178135,2.794118,1.500000,42.6,25292.0,7254.0,317
2,*324973/22_5308,3057820030,*324973/22,5308,462 60TH STREET,FOURTH FLOOR APT AKA,2024-08-13,BROOKLYN,11220,Not an Ejectment,Possession,40.640008,-74.017068,7.0,38.0,122.0,3143435,Sunset Park West,2024,2024-08,POINT (-74.017068 40.640008),1907.0,C3,4.0,4.0,"LIN, RONG LAN",4800.0,pre-war,walk-up,False,mid-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",3-5 units,False,medium-small,Q4 (largest 25%),1900-1909,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7

In [90]:
covid.head()

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.9870,0.9470,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.200000,93.6,26048.0,1266.0,124
1,0050153/20_106030,4031560133,0050153/20,106030,98-05 67TH AVENUE,12F,2022-04-14,QUEENS,11375,Not an Ejectment,Possession,40.724241,-73.855552,6.0,29.0,71306.0,4074666,Forest Hills,2022,2022-04,POINT (-73.855552 40.724241),1960.0,D3,13.0,181.0,MARSEILLES LEASING LIMITED PARTNERSHIP,177710.0,post-war,elevator,False,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,11375,75212.0,0.4759,0.5698,0.8789,0.8057,0.7322,12.0,4.8,6.1,3.7,20.4,18.0,10.5,7.9,41.9,5.8,25.4,2.7,16.4,28.5,0.1,0.0,4.6,0.7,53.0,47.0,False,Q1 (Low),0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,62.0,0.0,34.0,0.0,0.0,4.0,1.0,0.0,0.0,2.0,5.0,112.0,0.001105,0.2,6.000000,0.079775,17,3077.0,0.025926,0.014359,0.009573,0.008376,0.003829,0.016274,0.015237,0.020263,3.529412,0.003315,19.1,32853.0,6340.0,11
2,0052002/19_101926,3051370021,0052002/19,101926,199 VERONICA PLACE,1ST FLOOR,2020-03-02,BROOKLYN,11226,Not an Ejectment,Possession,40.645404,-73.952578,17.0,40.0,792.0,3117969,Erasmus,2020,2020-03,POINT (-73.952578 40.645404),1920.0,B3,2.0,2.0,"AANS, LLC.",1496.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q2 (25-50%),1920-1929,11226,101053.0,0.9300,0.4536,0.9639,0.9692,0.9220,23.7,5.9,13.9,9.1,13.1

In [91]:
normal.isna().sum()

,0
primary_key,0
bbl,0
court_index_number,0
docket_number,0
eviction_address,0
...,...
average_5year_eviction_count_per_unit(building),0
bh_pct,0
unit_count_zipcode,5
building_count_zipcode,5


In [92]:
normal.loc[normal['zipcode'] == 10162, 'unit_count_zipcode'] = 224
normal.loc[normal['zipcode'] == 10162, 'building_count_zipcode'] = 1

In [93]:
11359 in bbl.zipcode.unique()

False

In [127]:
nan_rows = normal[normal.isna().any(axis=1)]  #
nan_rows[['zipcode']]
# 10162 1 building 224 units
# 10359 --> 10360 (doesn)
# 11430 is JFK

,zipcode


In [95]:
nan_rows = covid[covid.isna().any(axis=1)]  #
nan_rows[['zipcode']]
# 10162 1 building 224 units
# 10359 --> 10360 (doesn)
# 11430 is JFK

,zipcode
3110,10162


In [96]:
covid.loc[covid['zipcode'] == 10162, 'unit_count_zipcode'] = 224
covid.loc[covid['zipcode'] == 10162, 'building_count_zipcode'] = 1

In [97]:
zipcodes_to_drop = [11359, 11430]
normal = normal[~normal['zipcode'].isin(zipcodes_to_drop)]

In [98]:
normal.columns[-30:]

Index(['noise_complaints', 'other_issues', 'pest_issues', 'plumbing_issues',
       'police_matters', 'public_nuisance', 'safety_concerns',
       'sanitation_issues', 'walls_ceilings', 'total_complaints',
       'average_year_eviction_count_per_unit(building)',
       'average_year_eviction_count(building)',
       'average_year_eviction_count_per_nta(nta)',
       'evictions_per_nta_1kpop(nta)', 'buildings_affected_per_nta',
       'unit_per_nta', 'evictions_per_1kunit_nta_1kpop(nta)',
       'children_impacted_nta_per_1kpop', 'poverty_impacted_nta_per_1kpop',
       'disability_impacted_nta_per_1kpop',
       'unemployed_impacted_nta_per_1kpop', 'elderly_impacted_nta_per_1kpop',
       'bh_impacted_nta_per_1kpop', 'hburd_impacted_nta_per_1kpop',
       'average_year_eviction_per_building_nta(nta)',
       'average_5year_eviction_count_per_unit(building)', 'bh_pct',
       'unit_count_zipcode', 'building_count_zipcode',
       'average_year_eviction_count_per_zipcode(z)'],
      dtyp

In [99]:
normal[normal['zipcode'] == 10162]

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z)
1756,084970/16_202073,1014880001,084970/16,202073,500 EAST 77TH STREET,325,2018-03-20,MANHATTAN,10162,Not an Ejectment,Possession,40.769975,-73.951186,8.0,5.0,132.0,1046407,Lenox Hill-Roosevelt Island,2018,2018-03,POINT (-73.951186 40.769975),1964.0,D3,17.0,210.0,EAST 77TH REALTY LLC,257247.0,post-war,elevator,False,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,True,mega,Q4 (largest 25%),1960-1969,10162,1523.0,0.3819,0.2746,0.5234,0.4678,0.3905,27.2,0.0,6.0,0.0,59.0,13.5,19.8,0.0,54.5,0.3,46.2,0.6,9.9,2.9,0.0,0.0,0.3,0.0,13.7,86.3,False,Q1 (Low),low,0.0,0.0,0.0,0.0,4.0,1.0,4.0,0.0,0.0,0.0,7.0,1.0,60.0,1.0,5.0,3.0,0.0,0.0,0.0,6.0,0.0,92.0,0.002857,0.6,55.4,36.375575,188,39480.0,0.921367,4.910703,9.894156,7.202364,0.0,21.461589,3.819435,16.805515,2.946809,0.014286,10.5,224.0,1.0,3
14692,309061/22_210917,1014880001,309061/22,210917,500 EAST 77TH STREET,335,2023-08-15,MANHATTAN,10162,Not an Ejectment,Possession,40.769975,-73.951186,8.0,5.0,132.0,1046407,Lenox Hill-Roosevelt Island,2023,2023-08,POINT (-73.951186 40.769975),1964.0,D3,17.0,210.0,EAST 77TH REALTY LLC,257247.0,post-war,elevator,False,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,True,mega,Q4 (largest 25%),1960-1969,10162,1523.0,0.3819,0.2746,0.5234,0.4678,0.3905,27.2,0.0,6.0,0.0,59.0,13.5,19.8,0.0,54.5,0.3,46.2,0.6,9.9,2.9,0.0,0.0,0.3,0.0,13.7,86.3,False,Q1 (Low),low,0.0,0.0,0.0,0.0,4.0,1.0,4.0,0.0,0.0,0.0,7.0,1.0,60.0,1.0,5.0,3.0,0.0,0.0,0.0,6.0,0.0,92.0,0.002857,0.6,55.4,36.375575,188,39480.0,0.921367,4.910703,9.894156,7.202364,0.0,21.461589,3.819435,16.805515,2.946809,0.014286,10.5,224.0,1.0,3
39043,64622/19_493494,1014880001,64622/19,493494,500 EAST 77TH STREET,401 ##,2019-10-07,MANHATTAN,10162,Not an Ejectment,Possession,40.769975,-73.951186,8.0,5.0,132.0,1046407,Lenox Hill-Roosevelt Island,2019,2019-10,POINT (-73.951186 40.769975),1964.0,D3,17.0,210.0,EAST 77TH REALTY LLC,257247.0,post-war,elevator,False,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,True,me

In [100]:
normal.isna().sum().sum(), covid.isna().sum().sum()

(np.int64(0), np.int64(0))

In [101]:
# stop

# **average_year_eviction_per_building_zipcode(z)**

In [102]:
normal['average_year_eviction_per_building_zipcode(z)'] = normal['average_year_eviction_count_per_zipcode(z)']/normal['building_count_zipcode']
covid['average_year_eviction_per_building_zipcode(z)'] = covid['average_year_eviction_count_per_zipcode(z)']/covid['building_count_zipcode']
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964


In [103]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946


In [104]:
covid['average_year_eviction_per_building_zipcode(z)'].isna().sum(), normal['average_year_eviction_per_building_zipcode(z)'].isna().sum()

(np.int64(0), np.int64(0))

# **evictions_per_zipcode_1kpop(z)**

In [105]:
normal['evictions_per_zipcode_1kpop(z)'] = normal['average_year_eviction_count_per_zipcode(z)']/normal['e_totpop']*1000
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223


In [106]:
covid['evictions_per_zipcode_1kpop(z)'] = covid['average_year_eviction_count_per_zipcode(z)']/covid['e_totpop']*1000
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398


# **evictions_per_1kunit_zipcode_1kpop(nta)**

In [107]:
normal['evictions_per_1kunit_zipcode_1kpop(z)'] = normal['average_year_eviction_count_per_unit(building)']/normal['e_totpop']*1000
covid['evictions_per_1kunit_zipcode_1kpop(z)'] = covid['average_year_eviction_count_per_unit(building)']/covid['e_totpop']*1000
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223,0.002755


In [108]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398,0.000819


# **7 vulnerable groups impacted by evictions per zipcode**

# **children_impacted_zipcode_per_1kpop**

In [109]:
normal['children_impacted_zipcode_per_1kpop(z)'] = normal['evictions_per_zipcode_1kpop(z)']*normal['ep_age17']*0.01
covid['children_impacted_zipcode_per_1kpop(z)'] = covid['evictions_per_zipcode_1kpop(z)']*covid['ep_age17']*0.01
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223,0.002755,3.06815


In [110]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398,0.000819,0.402177


# **poverty_impacted_zipcode_per_1kpop**

In [111]:
normal['poverty_impacted_zipcode_per_1kpop(z)'] = normal['evictions_per_zipcode_1kpop(z)']*normal['ep_pov150']*0.01
covid['poverty_impacted_zipcode_per_1kpop(z)'] = covid['evictions_per_zipcode_1kpop(z)']*covid['ep_pov150']*0.01
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223,0.002755,3.06815,4.62268


In [112]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398,0.000819,0.402177,0.601742


# **disability_impacted_zipcode_per_1kpop**

In [113]:
normal['disability_impacted_zipcode_per_1kpop(z)'] = normal['evictions_per_zipcode_1kpop(z)']*normal['ep_disabl']*0.01
covid['disability_impacted_zipcode_per_1kpop(z)'] = covid['evictions_per_zipcode_1kpop(z)']*covid['ep_disabl']*0.01
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223,0.002755,3.06815,4.62268,1.881799


In [114]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398,0.000819,0.402177,0.601742,0.185855


# **unemployed_impacted_zipcode_per_1kpop**

In [115]:
normal['unemployed_impacted_zipcode_per_1kpop(z)'] = normal['evictions_per_zipcode_1kpop(z)']*normal['ep_unemp']*0.01
covid['unemployed_impacted_zipcode_per_1kpop(z)'] = covid['evictions_per_zipcode_1kpop(z)']*covid['ep_unemp']*0.01
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223,0.002755,3.06815,4.62268,1.881799,1.513621


In [116]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398,0.000819,0.402177,0.601742,0.185855,0.176714


# **elderly_impacted_zipcode_per_1kpop**


In [117]:
normal['elderly_impacted_zipcode_per_1kpop(z)'] = normal['evictions_per_zipcode_1kpop(z)']*normal['ep_age65']*0.01
covid['elderly_impacted_zipcode_per_1kpop(z)'] = covid['evictions_per_zipcode_1kpop(z)']*covid['ep_age65']*0.01
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z),elderly_impacted_zipcode_per_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223,0.002755,3.06815,4.62268,1.881799,1.513621,1.881799


In [118]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z),elderly_impacted_zipcode_per_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398,0.000819,0.402177,0.601742,0.185855,0.176714,0.170621


# **bh_impacted_zipcode_per_1kpop**


In [119]:
normal['bh_impacted_zipcode_per_1kpop(z)'] = normal['evictions_per_zipcode_1kpop(z)']*normal['bh_pct']*0.01
covid['bh_impacted_zipcode_per_1kpop(z)'] = covid['evictions_per_zipcode_1kpop(z)']*covid['bh_pct']*0.01
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z),elderly_impacted_zipcode_per_1kpop(z),bh_impacted_zipcode_per_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223,0.002755,3.06815,4.62268,1.881799,1.513621,1.881799,12.09533


In [120]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z),elderly_impacted_zipcode_per_1kpop(z),bh_impacted_zipcode_per_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398,0.000819,0.402177,0.601742,0.185855,0.176714,0.170621,1.4259


# **hburd_impacted_zipcode_per_1kpop**

In [121]:
normal['hburd_impacted_zipcode_per_1kpop(z)'] = normal['evictions_per_zipcode_1kpop(z)']*normal['ep_hburd']*0.01
covid['hburd_impacted_zipcode_per_1kpop(z)'] = covid['evictions_per_zipcode_1kpop(z)']*covid['ep_hburd']*0.01
normal.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z),elderly_impacted_zipcode_per_1kpop(z),bh_impacted_zipcode_per_1kpop(z),hburd_impacted_zipcode_per_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.914,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.93154,0.379211,0.305018,0.379211,2.437392,1.228314,3.668966,1.333333,88.7,30749.0,9431.0,1320,0.139964,13.636223,0.002755,3.06815,4.62268,1.881799,1.513621,1.881799,12.09533,6.095392


In [122]:
covid.head(1)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z),elderly_impacted_zipcode_per_1kpop(z),bh_impacted_zipcode_per_1kpop(z),hburd_impacted_zipcode_per_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.865396,-73.901317,7.0,14.0,265.0,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0,C0,3.0,3.0,MONJU SARKER,3420.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0,0.9954,0.9407,0.987,0.947,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,14.0,0.066667,0.2,18.666667,0.229329,38,114.0,2.011655,0.060543,0.090585,0.027978,0.026602,0.025685,0.214652,0.130029,4.912281,0.2,93.6,26048.0,1266.0,124,0.097946,1.523398,0.000819,0.402177,0.601742,0.185855,0.176714,0.170621,1.4259,0.863766


In [123]:
normal.columns[-31:]
# 12 nta, 12 zipcode, and 3 building, 2 nta agg, 2 zipcode agg.
# 11 + 11 + 2 + 3 + 2 + 1 = 30

Index(['total_complaints', 'average_year_eviction_count_per_unit(building)',
       'average_year_eviction_count(building)',
       'average_year_eviction_count_per_nta(nta)',
       'evictions_per_nta_1kpop(nta)', 'buildings_affected_per_nta',
       'unit_per_nta', 'evictions_per_1kunit_nta_1kpop(nta)',
       'children_impacted_nta_per_1kpop', 'poverty_impacted_nta_per_1kpop',
       'disability_impacted_nta_per_1kpop',
       'unemployed_impacted_nta_per_1kpop', 'elderly_impacted_nta_per_1kpop',
       'bh_impacted_nta_per_1kpop', 'hburd_impacted_nta_per_1kpop',
       'average_year_eviction_per_building_nta(nta)',
       'average_5year_eviction_count_per_unit(building)', 'bh_pct',
       'unit_count_zipcode', 'building_count_zipcode',
       'average_year_eviction_count_per_zipcode(z)',
       'average_year_eviction_per_building_zipcode(z)',
       'evictions_per_zipcode_1kpop(z)',
       'evictions_per_1kunit_zipcode_1kpop(z)',
       'children_impacted_zipcode_per_1kpop(z)',
   

In [124]:
covid.columns[-31:]

Index(['total_complaints', 'average_year_eviction_count_per_unit(building)',
       'average_year_eviction_count(building)',
       'average_year_eviction_count_per_nta(nta)',
       'evictions_per_nta_1kpop(nta)', 'buildings_affected_per_nta',
       'unit_per_nta', 'evictions_per_1kunit_nta_1kpop(nta)',
       'children_impacted_nta_per_1kpop', 'poverty_impacted_nta_per_1kpop',
       'disability_impacted_nta_per_1kpop',
       'unemployed_impacted_nta_per_1kpop', 'elderly_impacted_nta_per_1kpop',
       'bh_impacted_nta_per_1kpop', 'hburd_impacted_nta_per_1kpop',
       'average_year_eviction_per_building_nta(nta)',
       'average_3year_eviction_count_per_unit(building)', 'bh_pct',
       'unit_count_zipcode', 'building_count_zipcode',
       'average_year_eviction_count_per_zipcode(z)',
       'average_year_eviction_per_building_zipcode(z)',
       'evictions_per_zipcode_1kpop(z)',
       'evictions_per_1kunit_zipcode_1kpop(z)',
       'children_impacted_zipcode_per_1kpop(z)',
   

In [128]:
normal.isna().sum().sum(), covid.isna().sum().sum()

(np.int64(0), np.int64(0))

In [129]:
normal.to_csv('/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_svi_zipcode.csv', index=False)
covid.to_csv('/content/drive/My Drive/X999/bbl_evictions_311_svi_covid_svi_zipcode.csv', index=False)